In [3]:
import os
import pymongo
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

MONGO_DB_USER = os.environ["MONGO_DB_USER"]
MONGO_DB_PASSWORD = os.environ["MONGO_DB_PASSWORD"]
MONGO_DB_HOST = os.environ["MONGO_DB_HOST"]
MONGO_DB_PORT = os.environ["MONGO_DB_PORT"]

client = pymongo.MongoClient(
    f"mongodb://{MONGO_DB_USER}:{MONGO_DB_PASSWORD}@{MONGO_DB_HOST}:{MONGO_DB_PORT}/"
)

silver_db = client.silver
silver_events = silver_db.events
silver_games = silver_db.games

In [130]:
print("Most recent event in the Database:")

most_recent_event = silver_events.find_one(sort=[("date", -1)])

print(most_recent_event["date"])

Most recent event in the Database:
2024-07-09 00:00:00


## Biggest Winners

We'll be analysing the Double Tournament era here, which stated in 01.02.2024 according to Chess.com.

Source: https://www.chess.com/article/view/titled-tuesday

In [151]:
winners = silver_events.aggregate(
    [
        {
            "$match": {
                "date": {"$gte": datetime(2022, 1, 31)},
            }
        },
        {
            "$project": {
                "results": {
                    "$filter": {
                        "input": "$results",
                        "as": "r",
                        "cond": {"$eq": ["$$r.position", 1]},
                    }
                },
                "date": 1,
            }
        },
    ]
)

dict_winners = {}

for i in winners:
    for winner in i["results"]:
        dict_winners.setdefault(winner["player"], 0)
        dict_winners[winner["player"]] += 1

print(f"Number of Events: {sum(dict_winners.values())}\n")

print(f"Players with most wins")
for key, value in sorted(dict_winners.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"{key:>22}: {value}")

Number of Events: 244

Players with most wins:
      Nakamura, Hikaru: 39
                Hikaru: 16
     Andreikin, Dmitry: 13
       Carlsen, Magnus: 12
         Dubov, Daniil: 11
    Fedoseev, Vladimir: 6
     Bortnyk, Olexandr: 6
        Xiong, Jeffery: 5
     Firouzja, Alireza: 5
   Duda, Jan-Krzysztof: 5


#### Notes

The following players have multiple accounts in the winners list and will be summed together:

- Hikaru Nakamura
- Magnus Carlsen
- Alireza Firouzja
- Aleksei Sarana
- Andreikin, Dmitry



In [152]:
dict_multiple_accounts = {
    "Nakamura, Hikaru": "Hikaru",
    "Carlsen, Magnus": "MagnusCarlsen",
    "Firouzja, Alireza": "Firouzja2003",
    "Sarana, Alexey": "mishanick",
    "Andreikin, Dmitry": "FairChess_on_YouTube",
    "Duda, Jan-Krzysztof": "Polish_fighter3000",
}

for key, value in dict_multiple_accounts.items():
    dict_winners[key] += dict_winners[value]
    del dict_winners[value]

print(f"Players with most wins")
for key, value in sorted(dict_winners.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"{key:>22}: {value}")

Number of Events: 244

Players with most wins:
      Nakamura, Hikaru: 55
     Andreikin, Dmitry: 16
       Carlsen, Magnus: 16
         Dubov, Daniil: 11
     Firouzja, Alireza: 9
        Sarana, Alexey: 8
   Duda, Jan-Krzysztof: 8
    Fedoseev, Vladimir: 6
     Bortnyk, Olexandr: 6
        Xiong, Jeffery: 5


## Biggest upsets

An upset is when one player's rating is lower than the other player's.

Here we'll be looking into which were the biggest upsets in the Double-Tournament Era.

In [166]:
upsets = (
    silver_games.find(
        {
            "WhiteElo": {"$gt": 1000},
            "BlackElo": {"$gt": 1000},
            "Date": {"$gte": datetime(2022, 1, 1)},
        }
    )
    .limit(10)
    .sort({"Upset": 1})
)

upsets = pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

upsets.loc[:, "Chances of Upset %"] = 1 / (1 + 10 ** (-upsets["Upset"] / 400))

upsets.style.format({"Chances of Upset %": "{:.4%}", "Upset": "{:.0f}"})

,Event,Upset,White,WhiteElo,Black,BlackElo,Chances of Upset %
0,Titled Tuesday Blitz December 20 Early 2022,-1619,Susal_De_Silva,2808,"Kazgildy, Fariza",1189,0.0090%
1,Titled Tuesday Blitz October 18 Late 2022,-1546,"Eisa Mohmood, Alkhoori",1245,"Bordi, Kevin",2791,0.0136%
2,Titled Tuesday Blitz December 20 Early 2022,-1536,Kopylov_Alex,2790,"Mone, Ved",1254,0.0145%
3,Titled Tuesday Blitz December 20 Early 2022,-1463,The_Machine04,2652,"Kazgildy, Fariza",1189,0.0220%
4,Titled Tuesday Blitz April 26 Early 2022,-1414,"Bashirli, Saadat",1173,"Bachmann, Axel",2587,0.0292%
5,Titled Tuesday Blitz December 20 Early 2022,-1393,"Benamar, Benious",1136,lemayvega,2529,0.0329%
6,Titled Tuesday Blitz December 20 Early 2022,-1350,CDOEspiao,2486,"Benamar, Benious",1136,0.0422%
7,Titled Tuesday Blitz December 20 Early 2022,-1328,"Castro Andrade, Seirbert Cristian",1364,"Xiong, Jeffery",2692,0.0478%
8,Titled Tuesday Blitz December 20 Early 2022,-1311,"Yildiz, Ayse Emek",1702,OparinGrigoriy,3013,0.0528%
9,Titled Tuesday Blitz January 03 Late 2023,-1309,"Gaehwiler, Gabriel",2404,"Eisa Mohmood, Alkhoori",1095,0.0534%


### Notes

In the tables above we can see that the biggest upset in this tournament had an elo difference of 1619, this had a chance of happening of just 0.009% ([Source](https://fivethirtyeight.com/features/introducing-nfl-elo-ratings/)).

### Carlsen and Nakamura Upsets

In [179]:
print(
    f"Number of games player by Carlsen: {silver_games.count_documents({'$or': [{'White': 'Carlsen, Magnus'}, {'Black': 'Carlsen, Magnus'}]})}"
)

upsets = (
    silver_games.find(
        {
            "$or": [
                {"White": {"$in": ["Carlsen, Magnus"]}},
                {"Black": {"$in": ["Carlsen, Magnus"]}},
            ],
            "WhiteElo": {"$gt": 1000},
            "BlackElo": {"$gt": 1000},
        }
    )
    .limit(5)
    .sort({"Upset": 1})
)

carlsen_upsets = pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

carlsen_upsets.loc[:, "Chances of Upset %"] = 1 / (1 + 10 ** (-carlsen_upsets["Upset"] / 400))

carlsen_upsets.style.format({"Chances of Upset %": "{:.4%}", "Upset": "{:.0f}"})

Number of games player by Carlsen: 457


,Event,Upset,White,WhiteElo,Black,BlackElo,Chances of Upset %
0,Titled Tuesday Blitz March 21 Early 2023,-475,"Carlsen, Magnus",2852,"Tillyaev, Ulugbek",2377,6.0978%
1,Titled Tuesday Blitz January 31 Late 2023,-440,"Erdogdu, Mert",2413,"Carlsen, Magnus",2853,7.3588%
2,Titled Tuesday Blitz May 9 Late 2023,-391,"Carlsen, Magnus",2852,"Makarian, Rudik",2461,9.5282%
3,Titled Tuesday Blitz July 25 Late 2023,-355,"Carlsen, Magnus",3256,"Dardha, Daniel",2901,11.4706%
4,Titled Tuesday Blitz February 28 Late 2023,-352,"Pranav, V",2500,"Carlsen, Magnus",2852,11.6472%


In [180]:
print(
    f"Number of games player by Nakamura: {silver_games.count_documents({'$or': [{'White': 'Nakamura, Hikaru'}, {'Black': 'Nakamura, Hikaru'}]})}"
)

upsets = (
    silver_games.find(
        {
            "$or": [
                {"White": {"$in": ["Nakamura, Hikaru"]}},
                {"Black": {"$in": ["Nakamura, Hikaru"]}},
            ],
            "WhiteElo": {"$gt": 1000},
            "BlackElo": {"$gt": 1000},
        }
    )
    .limit(5)
    .sort({"Upset": 1})
)

nakamura_upsets = pd.DataFrame(upsets)[["Event", "Upset", "White", "WhiteElo", "Black", "BlackElo"]]

nakamura_upsets.loc[:, "Chances of Upset %"] = 1 / (1 + 10 ** (-nakamura_upsets["Upset"] / 400))

nakamura_upsets.style.format({"Chances of Upset %": "{:.4%}", "Upset": "{:.0f}"})

Number of games player by Nakamura: 1664


,Event,Upset,White,WhiteElo,Black,BlackElo,Chances of Upset %
0,Titled Tuesday Blitz August 15 Early 2023,-1180,"Nakamura, Hikaru",3255,"Nugumanov, Bakhtiyar",2075,0.1121%
1,Titled Tuesday Blitz May 10 Early 2022,-1054,"Nakamura, Hikaru",2850,"Aslanov, Umid",1796,0.2312%
2,Titled Tuesday Blitz August 15 Early 2023,-761,"Benedetti, Julio",2510,"Nakamura, Hikaru",3271,1.2362%
3,Titled Tuesday Blitz June 6 Late 2023,-759,"Nakamura, Hikaru",2874,"Hajiyev, Kanan",2115,1.2504%
4,Titled Tuesday Blitz May 10 Early 2022,-656,"Stoleriu, George",2194,"Nakamura, Hikaru",2850,2.2396%


### Notes on Nakamura and Carlsen Upsets

Magnus Carlsen never had a huge upset, having lost to someone with a rating difference of 475 and a chance of upset of 6%.

Hikaru Nakamura had more upsets, losing many times to players with over 500 difference in rating and chacnce of upsets below 1%.